In [3]:
!pip install -U torchtext==0.10.0 --quiet

### Import Libraries for Data Engineering

In [55]:
import torch
from torch import nn
import torch.optim as optim
from torchtext.legacy import data
from torchtext.legacy import datasets
# import torch.nn.functional as F

import numpy as np

import random

In [56]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Tokenizer definition

In [13]:
!python -m spacy download en_core_web_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm', batch_first=True)
LABEL = data.LabelField(dtype = torch.float, batch_first=True)

In [62]:
## import data
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


d:\Saeed Ahmad Work\Github Work\machine_learning\mini_projects\movies_reviews_classification\.data\imdb\aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.1MB/s]


In [63]:
print(vars(train_data.examples[0]))

{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'l

In [64]:
train_data, valid_data = train_data.split(split_ratio=0.7, stratified=True, random_state = random.seed(SEED))

print(f'Number of training examples   : {len(train_data)}')
print(f'Number of validation examples : {len(valid_data)}')
print(f'Number of testing examples    : {len(test_data)}')

Number of training examples   : 17500
Number of validation examples : 7500
Number of testing examples    : 25000


### Build Vocabulary

In [65]:
MAX_VOCAB_SIZE = 20000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)

LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary : {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

print(TEXT.vocab.freqs.most_common(20))

print(TEXT.vocab.itos[:10])

print(LABEL.vocab.stoi)

Unique tokens in TEXT vocabulary : 20002
Unique tokens in LABEL vocabulary: 2
[('the', 202689), (',', 192447), ('.', 165048), ('and', 109188), ('a', 109040), ('of', 100559), ('to', 93907), ('is', 75853), ('in', 61324), ('I', 53964), ('it', 53467), ('that', 49110), ('"', 43651), ("'s", 43059), ('this', 42281), ('-', 36976), ('/><br', 35313), ('was', 35289), ('as', 30401), ('with', 29831)]
['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
defaultdict(None, {'neg': 0, 'pos': 1})


### Data loader

In [66]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

print('Number of minibatch for training dataset   : {}'.format(len(train_iterator)))
print('Number of minibatch for validation dataset : {}'.format(len(valid_iterator)))
print('Number of minibatch for testing dataset    : {}'.format(len(test_iterator)))

Number of minibatch for training dataset   : 274
Number of minibatch for validation dataset : 118
Number of minibatch for testing dataset    : 391


### LSTM model 

In [67]:
# Set Hyperparameters

embedding_dim = 256
hidden_units = 128
EPOCHS = 50
learning_rate = 5e-4


# Build NN model
class LSTM(nn.Module):
    def __init__(self, vocab_size, hidden_dim, output_dim, embedding_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, _ = self.rnn(embedded)
        output = self.linear(output[:, -1, :])
        return output
  
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

model = LSTM(len(TEXT.vocab), 128, len(LABEL.vocab)-1, 300, 0.2)
model

c:\Users\root\anaconda3\envs\gpu-env\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


LSTM(
  (embedding): Embedding(20002, 300, padding_idx=0)
  (rnn): LSTM(300, 128, batch_first=True, dropout=0.2)
  (linear): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [68]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,220,889 trainable parameters


In [69]:

## Optimizer and Loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()

## change model and criterion to device
model = model.to(device)
criterion = criterion.to(device)

In [70]:
'''
M7. Define Accuracy Function
'''
def binary_accuracy(preds, target):
    '''
    from https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
    '''
    # round predictions to the closest integer (0 or 1)
    rounded_preds = torch.round(torch.sigmoid(preds))

    #convert into float for division
    correct = (rounded_preds == target).float()

    # rounded_preds = [ 1   0   0   1   1   1   0   1   1   1]
    # targets       = [ 1   0   1   1   1   1   0   1   1   0]
    # correct       = [1.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0]
    acc = correct.sum() / len(correct)
    return acc

### Define Training Function

In [71]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        # We initialize the gradient to 0 for every batch.
        optimizer.zero_grad()

        # batch of sentences 
        predictions = model(batch.text).squeeze(1)
        
        # Calculate the loss value by comparing the prediction result with batch.label 
        loss = criterion(predictions, batch.label)

        # Accuracy calculation
        acc = binary_accuracy(predictions, batch.label)
        
        # Backpropagation using backward()
        loss.backward()

        # Update the parameters using the optimization algorithm
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Define Evaluation Function

In [72]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # "evaluation mode" : turn off "dropout" or "batch nomalizaation"
    model.eval()

    # Use less memory and speed up computation by preventing gradients from being computed in pytorch
    with torch.no_grad():
    
        for batch in iterator:
            
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Model Training

In [73]:
import time
import os

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [74]:

best_valid_loss = float('inf')

'''
Episode / each step Process
'''
for epoch in range(EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), os.path.join('.models', f'model_{epoch}.pt'))
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 26m 45s
	Train Loss: 0.695 | Train Acc: 49.74%
	 Val. Loss: 0.689 |  Val. Acc: 49.74%
Epoch: 02 | Epoch Time: 25m 42s
	Train Loss: 0.693 | Train Acc: 50.13%
	 Val. Loss: 0.687 |  Val. Acc: 56.06%
Epoch: 03 | Epoch Time: 25m 42s
	Train Loss: 0.693 | Train Acc: 50.46%
	 Val. Loss: 0.684 |  Val. Acc: 54.88%
Epoch: 04 | Epoch Time: 26m 12s
	Train Loss: 0.692 | Train Acc: 49.86%
	 Val. Loss: 0.700 |  Val. Acc: 54.36%
Epoch: 05 | Epoch Time: 20m 53s
	Train Loss: 0.689 | Train Acc: 51.09%
	 Val. Loss: 0.697 |  Val. Acc: 49.67%
Epoch: 06 | Epoch Time: 18m 31s
	Train Loss: 0.689 | Train Acc: 50.21%
	 Val. Loss: 0.689 |  Val. Acc: 57.19%
Epoch: 07 | Epoch Time: 18m 6s
	Train Loss: 0.688 | Train Acc: 50.64%
	 Val. Loss: 0.700 |  Val. Acc: 53.18%
Epoch: 08 | Epoch Time: 16m 58s
	Train Loss: 0.688 | Train Acc: 49.87%
	 Val. Loss: 0.693 |  Val. Acc: 55.88%
Epoch: 09 | Epoch Time: 17m 12s
	Train Loss: 0.687 | Train Acc: 50.16%
	 Val. Loss: 0.698 |  Val. Acc: 56.69%
Epoch: 10 |

In [75]:
model.load_state_dict(torch.load(os.path.join('.models', 'model_23.pt')))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.424 | Test Acc: 84.14%


In [76]:
'''
Training result test for Code Engineering
'''
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

examples = [
  "This film is terrible",
  "This film is great",
  "This movie is fantastic"
]

for idx in range(len(examples)) :

    sentence = examples[idx]
    pred = predict_sentiment(model,sentence)
    print("\n",sentence)
    if pred >= 0.5 :
        print(f">>>This is a positive review. ({pred : .2f})")
    else:
        print(f">>>This is a negative review.({pred : .2f})")


 This film is terrible
>>>This is a negative review.( 0.43)

 This film is great
>>>This is a positive review. ( 0.93)

 This movie is fantastic
>>>This is a positive review. ( 0.86)
